<a href="https://colab.research.google.com/github/Kanyinsola-O/CE888/blob/main/Lab7/Lab7p2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras import layers
from keras import models

In [2]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


In [3]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                3

In [4]:
from keras.datasets import mnist
from keras.utils import to_categorical

In [5]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255
test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

11493376/11490434 [==============================] - 0s 0us/step


In [6]:
model.compile(optimizer='rmsprop', 
              loss='categorical_crossentropy',
              metrics=['accuracy'])
history = model.fit(train_images, train_labels, epochs=5, batch_size=64, validation_split=0.2)

Epoch 1/5
750/750 [==============================] - 44s 58ms/step - loss: 0.4654 - accuracy: 0.8496 - val_loss: 0.0623 - val_accuracy: 0.9816
Epoch 2/5
750/750 [==============================] - 43s 57ms/step - loss: 0.0589 - accuracy: 0.9808 - val_loss: 0.0682 - val_accuracy: 0.9807
Epoch 3/5
750/750 [==============================] - 43s 58ms/step - loss: 0.0368 - accuracy: 0.9887 - val_loss: 0.0449 - val_accuracy: 0.9875
Epoch 4/5
750/750 [==============================] - 43s 57ms/step - loss: 0.0277 - accuracy: 0.9917 - val_loss: 0.0398 - val_accuracy: 0.9896
Epoch 5/5
750/750 [==============================] - 43s 57ms/step - loss: 0.0210 - accuracy: 0.9935 - val_loss: 0.0468 - val_accuracy: 0.9893


##Model Evaluation

In [7]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
test_acc

313/313 [==============================] - 3s 10ms/step - loss: 0.0304 - accuracy: 0.9906


0.9905999898910522

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

##T1

In [ ]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import GridSearchCV

def get_model(act='relu', opt='rmsprop'):
  '''Returns a cnn model with parameters:
      act = activation function to be evaluated for hidden/input layers
      opt = optimzer to be evaluated '''
  model = models.Sequential()
  model.add(layers.Conv2D(32, (3, 3), activation=act, input_shape=(28, 28, 1)))
  model.add(layers.MaxPooling2D((2, 2)))

  model.add(layers.Conv2D(64, (3, 3), activation=act))
  model.add(layers.MaxPooling2D((2, 2)))
  
  model.add(layers.Conv2D(64, (3, 3), activation=act))
  model.add(layers.Flatten())
  
  model.add(layers.Dense(64, activation=act))
  model.add(layers.Dense(10, activation='softmax'))

  model.compile(optimizer=opt, 
              loss='categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

#Set of parameters to evaluate, 3 activation functions and 3 optimizers
params = {'act':  ['relu', 'tanh', 'linear'],
          'opt' : ['rmsprop', 'adam', 'sgd']}

#Uses Kerasclassifier wrapper to build a model object to fit to SKlearns GridSarchCV
model = KerasClassifier(build_fn=get_model, epochs=5, batch_size=64,
                        validation_split=0.2)


grid = GridSearchCV(model, params, cv=3, n_jobs=1)
results = grid.fit(train_images, train_labels)

print(f'Optimal accuracy results: {results.best_score_} with parameters: {results.best_params_} ')

#print the results of all tested optimizers and activation functions

means = results.cv_results_['mean_test_score']
params = results.cv_results_['params']

Epoch 1/5
500/500 [==============================] - 30s 58ms/step - loss: 0.5821 - accuracy: 0.8177 - val_loss: 0.0729 - val_accuracy: 0.9776
Epoch 2/5
500/500 [==============================] - 29s 58ms/step - loss: 0.0717 - accuracy: 0.9766 - val_loss: 0.0621 - val_accuracy: 0.9819
Epoch 3/5
500/500 [==============================] - 29s 58ms/step - loss: 0.0414 - accuracy: 0.9865 - val_loss: 0.0479 - val_accuracy: 0.9861
Epoch 4/5
500/500 [==============================] - 29s 58ms/step - loss: 0.0321 - accuracy: 0.9892 - val_loss: 0.0454 - val_accuracy: 0.9869
Epoch 5/5
313/313 [==============================] - 6s 18ms/step - loss: 0.0475 - accuracy: 0.9869
Epoch 1/5
500/500 [==============================] - 30s 58ms/step - loss: 0.5492 - accuracy: 0.8252 - val_loss: 0.0768 - val_accuracy: 0.9758
Epoch 2/5
500/500 [==============================] - 29s 59ms/step - loss: 0.0717 - accuracy: 0.9790 - val_loss: 0.0504 - val_accuracy: 0.9849
Epoch 3/5
500/500 [=======================

In [ ]:
print(f'Optimal accuracy results: {results.best_score_} with parameters: {results.best_params_} ')
for x, y in zip(means,params):
  print(f'{y} gave {x}')